In [1]:
!mkdir -p ~/.cloudvolume/secrets
!cp /notebooks/*.json ~/.cloudvolume/secrets/
!pip install cloud-volume
!pip install scikit-image

    100% |################################| 71kB 2.2MB/s ta 0:00:011
    100% |################################| 51kB 5.8MB/s ta 0:00:011
    100% |################################| 143kB 3.6MB/s ta 0:00:01
    100% |################################| 92kB 4.1MB/s ta 0:00:011
    100% |################################| 419kB 2.3MB/s ta 0:00:01
    100% |################################| 194kB 3.7MB/s ta 0:00:01
    100% |################################| 71kB 7.9MB/s eta 0:00:01
    100% |################################| 81kB 6.2MB/s ta 0:00:011
    100% |################################| 7.1MB 211kB/s eta 0:00:01
    100% |################################| 92kB 12.1MB/s ta 0:00:01
    100% |################################| 133kB 8.9MB/s eta 0:00:01
    100% |################################| 143kB 9.7MB/s eta 0:00:01
    100% |################################| 51kB 10.4MB/s ta 0:00:01
    100% |################################| 92kB 11.4MB/s ta 0:00:01
    100% |#####################

    100% |################################| 51kB 11.7MB/s ta 0:00:01
    100% |################################| 143kB 9.7MB/s eta 0:00:01
    100% |################################| 81kB 13.0MB/s ta 0:00:01
    100% |################################| 71kB 12.6MB/s ta 0:00:01
    100% |################################| 61kB 12.9MB/s ta 0:00:01
    100% |################################| 61kB 12.0MB/s ta 0:00:01
    100% |################################| 51kB 11.2MB/s ta 0:00:01
    100% |################################| 92kB 11.2MB/s ta 0:00:01
    100% |################################| 153kB 8.9MB/s eta 0:00:01
    100% |################################| 61kB 12.0MB/s ta 0:00:01
    100% |################################| 92kB 12.9MB/s ta 0:00:01
    100% |################################| 51kB 7.0MB/s eta 0:00:01
    100% |################################| 51kB 12.4MB/s ta 0:00:01
    100% |################################| 61kB 11.1MB/s ta 0:00:01
    100% |######################

    100% |################################| 61kB 11.8MB/s ta 0:00:01
    100% |################################| 143kB 8.9MB/s eta 0:00:01
    100% |################################| 4.3MB 359kB/s eta 0:00:01
    100% |################################| 61kB 11.9MB/s ta 0:00:01
    100% |################################| 51kB 11.8MB/s ta 0:00:01
    100% |################################| 81kB 12.3MB/s ta 0:00:01
    100% |################################| 102kB 12.3MB/s a 0:00:01
    100% |################################| 204kB 6.7MB/s eta 0:00:01
    100% |################################| 552kB 2.8MB/s eta 0:00:01
    100% |################################| 81kB 12.1MB/s ta 0:00:01
    100% |################################| 9.0MB 173kB/s eta 0:00:01
    100% |################################| 163kB 8.6MB/s eta 0:00:01
    100% |################################| 829kB 2.0MB/s eta 0:00:01
    100% |################################| 71kB 13.0MB/s ta 0:00:01


    100% |################################| 225kB 6.9MB/s eta 0:00:01
  Running setup.py bdist_wheel for psutil ... done
  Stored in directory: /root/.cache/pip/wheels/fe/33/84/baea3d665de2d1af7e8f827f3883811bba5e4149443ccf8191
  Running setup.py bdist_wheel for tenacity ... done
  Stored in directory: /root/.cache/pip/wheels/cf/dd/89/99693d2e2c5954ccd7b7ba4fe9571b385712939d25e584a03a
  Running setup.py bdist_wheel for posix-ipc ... done
  Stored in directory: /root/.cache/pip/wheels/5a/5c/17/d0e2d421abaf3b4097bf9db11108380d734195c6d15c24269d
  Running setup.py bdist_wheel for inflection ... done
  Stored in directory: /root/.cache/pip/wheels/9f/fa/4f/ef45a51543bd855e40494298824ad5e750109ad18fa80278ad
  Running setup.py bdist_wheel for pandocfilters ... done
  Stored in directory: /root/.cache/pip/wheels/fc/a0/4c/ec26eaa3f2253144e29941bd4031c317a69595dd0b665a0779
  Running setup.py bdist_wheel for Markdown ... done
  Stored in directory: /root/.cache/pip/wheels/34/2d/c2/75f4fd57c1b79e5

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |################################| 25.3MB 59kB/s  eta 0:00:01
    100% |################################| 634kB 2.6MB/s eta 0:00:01
    100% |################################| 1.6MB 881kB/s eta 0:00:01
    100% |################################| 5.7MB 281kB/s eta 0:00:01
    100% |################################| 51kB 12.0MB/s ta 0:00:01
  Running setup.py bdist_wheel for networkx ... done
  Stored in directory: /root/.cache/pip/wheels/44/c0/34/6f98693a554301bdb405f8d65d95bbcd3e50180cbfdd98a94e
  Running setup.py bdist_wheel for toolz ... done
  Stored in directory: /root/.cache/pip/wheels/f4/0c/f6/ce6b2d1aa459ee97cc3c0f82236302bd62d89c86c700219463
Successfully built networkx toolz
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import os
import sys
import time
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import collections
import random

from cloudvolume import CloudVolume
from cloudvolume.lib import Vec, Bbox
from skimage.transform import resize

class Stack():
  """Collection of CloudVolume paths, with load & save operations
  """

  def __init__(self, paths):
    self.paths = paths

  def load(self, key, bbox, src_mip=0, dst_mip=0):
    src_cv = CloudVolume(self.paths[key], mip=src_mip, fill_missing=True)
    src_bbox = Bbox.from_slices(src_cv.slices_from_global_coords(bbox))
    img = src_cv[src_bbox.to_slices()][:,:,0,0]
    if dst_mip != src_mip:
        dst_cv = CloudVolume(self.paths[key], mip=dst_mip, fill_missing=True)
        dst_bbox = Bbox.from_slices(dst_cv.slices_from_global_coords(bbox))
        img = resize(img, dst_bbox.size3()[:2], preserve_range=True)
    return img

  def save(self, key, mip, bbox, img, is_field=False):
    """Save vector field to CloudVolume
    
    Args:
    * cv: CloudVolume object
    * mip: int representing mip-level of the CloudVolume
    * bbox: Bbox at mip 0 with size3 of (x,y,1)
    * img: nxm int64 ndarray with shape (x,y)
    * is_field: boolean whether img is a field object
        field object is 4d tensor with shape (1,x,y,2)
    """
    src_cv = CloudVolume(self.paths['image'], mip=mip)
    info = src_cv.info
    if is_field:
        info['data_type'] = 'float32'
        info['num_channels'] = 2
    cv = CloudVolume(self.paths[key], mip=mip, bounded=False, 
                     info=info, fill_missing=True, 
                     non_aligned_writes=True, cdn_cache=False)
    cv.commit_info()
    
    if is_field:
        s = cv.slices_from_global_coords(bbox) 
        s += (slice(0,2),)
        cv[s] = img.permute(1,2,0,3).data.cpu().numpy()
    else:
        s = cv.slices_from_global_coords(bbox)
        cv[s] = img[:,:,np.newaxis,np.newaxis]

class Optimizer():
  """
  
  Attributes:
    * src_image: nxm float64 ndarry normalized between [0,1]
      This is the image to be transformed by the returned vector field.
    * dst_images: list of nxm float64 ndarrays normalized between [0,1]
      This is the set of images that transformed src_image will be compared to.
    * src_mask_smooth: nxm float64 ndarray normalized between [0,1]
      The weight represents that degree to which a pixel participates in smooth
      deformation (0: not at all; 1: completely). This mask is used to reduce 
      the smoothness penalty for pixels that participate in a non-smooth 
      deformation.
    * src_mask_loss: nxm float64 ndarray normalized between [0,1]
      This mask is transformed by the vector field and used to reduce the MSE
      for pixels that participate in a defect.
    * dst_masks_loss: list of nxm float64 ndarrays normalized between [0,1]
      Exactly like the src_mask_loss above. These masks are only used to reduce the 
      MSE for pixels that participate in a defect.
  """
    
  def __init__(self, stack, bbox, dst_increment, image_mip, fold_mip, crack_mip, loss_mip, output_mip):
    self.stack = stack
    self.bbox = bbox
    self.image_mip = image_mip
    self.output_mip = output_mip
    self.src_image = stack.load('image', bbox, src_mip=image_mip, dst_mip=image_mip) / 255.0
    # self.src_fold = stack.load('fold', bbox, src_mip=fold_mip, dst_mip=image_mip) / 255.0
    # self.src_crack = stack.load('crack', bbox, src_mip=crack_mip, dst_mip=image_mip) / 255.0
    # self.src_smooth_mask = np.minimum(self.src_fold, self.src_crack)
    self.src_loss_mask = (stack.load('loss', bbox, src_mip=loss_mip, dst_mip=image_mip) == 0) * 1.0
    self.src_smooth_mask = (stack.load('loss', bbox, src_mip=loss_mip, dst_mip=image_mip) == 0) * 1.0
    
    self.dst_images = []
    self.dst_masks = []
    for z in dst_increment:
        dst_bbox = self.bbox + [0,0,z]
        dst_image = stack.load('image', dst_bbox, src_mip=image_mip, dst_mip=image_mip)
        self.dst_images.append(dst_image)
        dst_mask = (stack.load('loss', dst_bbox, src_mip=loss_mip, dst_mip=image_mip) == 0) * 1.0
        self.dst_masks.append(dst_mask)
        if output_mip != image_mip:
            dst_image = stack.load('image', dst_bbox, src_mip=output_mip, dst_mip=output_mip)
        self.stack.save('output', output_mip, dst_bbox, dst_image, is_field=False)
    
    self.identities = {}
        
  @staticmethod
  def center(var, dims, d):
      if not isinstance(d, collections.Sequence):
          d = [d for i in range(len(dims))]
      for idx, dim in enumerate(dims):
          if d[idx] == 0:
              continue
          var = var.narrow(dim, int(d[idx]/2), int(var.size()[dim] - d[idx]))
      return var

  def get_identity_grid(self, dim, cache=True):
      if dim not in self.identities:
          gx, gy = np.linspace(-1, 1, dim), np.linspace(-1, 1, dim)
          I = np.stack(np.meshgrid(gx, gy))
          I = np.expand_dims(I, 0)
          I = torch.FloatTensor(I)
          I = torch.autograd.Variable(I, requires_grad=False)
          I = I.permute(0,2,3,1).cuda()
          self.identities[dim] = I
      if cache:
          return self.identities[dim]
      else:
          return self.identities[dim].clone()

  def jacob(self, fields):
      def dx(f):
          p = Variable(torch.zeros((1,1,f.size(1),2))).cuda()
          return torch.cat((p, f[:,2:,:,:] - f[:,:-2,:,:], p), 1)
      def dy(f):
          p = Variable(torch.zeros((1,f.size(1),1,2))).cuda()
          return torch.cat((p, f[:,:,2:,:] - f[:,:,:-2,:], p), 2)
      fields = sum(map(lambda f: [dx(f), dy(f)], fields), [])
      field = torch.sum(torch.cat(fields, -1) ** 2, -1)
      return field

  def penalty(self, fields, mask=1):
      jacob = self.jacob(fields)
      jacob = torch.mul(jacob, mask)
      return torch.sum(jacob)

  def render(self, field):
      src = stack.load('image', bbox, src_mip=self.output_mip, dst_mip=self.output_mip) / 255.0
      src = torch.FloatTensor(src).cuda()
      src = Variable(src).unsqueeze(0).unsqueeze(0)
      if self.output_mip < self.image_mip:
          upsample = lambda x: nn.Upsample(scale_factor=2**x, mode='bilinear') if x >0 else (lambda y: y)
          field = upsample(self.image_mip - self.output_mip)(field.permute(0,3,1,2)).permute(0,2,3,1)
      y = F.grid_sample(src, field + self.get_identity_grid(field.size(2)))
      img = y.data.cpu().numpy()
      img = (img[0,0,:,:]*255).astype(np.uint8)
      self.stack.save('output', self.output_mip, self.bbox, img, is_field=False)

  def process(self, field=None, crop=0, write_field=True, ndownsamples=4, 
                                   currn=5, avgn=20, lambda1=0.4, lr=0.2, 
                                   eps=0.01, min_iter=20, max_iter=1000):
    """Compute vector field that minimizes mean squared error (MSE) between 
    transformed src_image & all dst_images regularized by the smoothness of the 
    vector field subject to masks that allow the vector field to not be smooth. 
    The minimization uses SGD.

    Args:
    * field: nxmx2 float64 torch tensor normalized between [0,1]
      This field represents the derived vector field that transforms the 
      src_image subject to the contraints of the minimization.
    
    Returns:
    * field: nxmx2 float64 torch tensor normalized between [0,1]
    """
    downsample = lambda x: nn.AvgPool2d(2**x,2**x, count_include_pad=False) if x > 0 else (lambda y: y)
    upsample = nn.Upsample(scale_factor=2, mode='bilinear')
    s = torch.FloatTensor(self.src_image)
    src_var = Variable((s - torch.mean(s)) / torch.std(s)).cuda().unsqueeze(0).unsqueeze(0)
    src_smooth_mask_var = Variable(torch.FloatTensor(self.src_smooth_mask)).cuda().unsqueeze(0).unsqueeze(0)
    src_loss_mask_var = Variable(torch.FloatTensor(self.src_loss_mask)).cuda().unsqueeze(0).unsqueeze(0)
    dst_vars = []
    dst_mask_vars = []
    for d, m in zip(self.dst_images, self.dst_masks):
      d = torch.FloatTensor(d)
      dst_var = Variable((d - torch.mean(d)) / torch.std(d)).cuda().unsqueeze(0).unsqueeze(0)
      dst_vars.append(dst_var)
      mask_var = Variable(torch.FloatTensor(m)).cuda().unsqueeze(0).unsqueeze(0)
      dst_mask_vars.append(mask_var)
      
    dim = int(src_var.size()[-1] / (2 ** (ndownsamples - 1)))
    if field is None:
        field = Variable(torch.zeros((1,dim,dim,2))).cuda().detach()
    else:
        field = Variable(field).cuda().detach()
    field.requires_grad = True
    updates = 0
    for k in reversed(range(ndownsamples)):
      src_ = downsample(k)(src_var).detach()
      src_.requires_grad = False
      src_smooth_mask_ = downsample(k)(src_smooth_mask_var).detach()
      src_smooth_mask_.requires_grad = False
      src_loss_mask_ = downsample(k)(src_loss_mask_var).detach()
      src_loss_mask_.requires_grad = False
      dst_list_, dst_mask_list_ = [], []
      for d, m in zip(dst_vars, dst_mask_vars):
        dst_ = downsample(k)(d).detach()
        dst_.requires_grad = False
        dst_list_.append(dst_)
        mask_ = downsample(k)(m).detach()
        mask_.requires_grad = False
        dst_mask_list_.append(mask_)
      field = field.detach()
      field.requires_grad = True
      opt = torch.optim.SGD([field], lr=lr/(k+1))
      #sched = lr_scheduler.StepLR(opt, step_size=1, gamma=0.995)
      costs = []
      start_updates = updates
      print('Downsample factor {0}x'.format(2**k))
      while True:
        updates += 1
        I = self.get_identity_grid(field.size(2))
        pred = F.grid_sample(src_, field + I)
        pred_mask = F.grid_sample(src_loss_mask_, field + I)
        # centered_mask = self.center(src_smooth_mask_.squeeze(0), (1,2), 128/(2**k))
        centered_mask = self.center(pred_mask.squeeze(0), (1,2), 128/(2**k))
        centered_field = self.center(field, (1,2), 128/(2**k))
        penalty1 = self.penalty([centered_field], centered_mask)
        cost = penalty1 * lambda1/(k+1)
        for d, m in zip(dst_list_, dst_mask_list_):
          mask = torch.mul(pred_mask, m)
          mse = torch.mul(pred - d, mask)**2
          cost += torch.mean(self.center(mse, (-1,-2), 128 / (2**k)))
        #cost = diff + penalty1 * lambda1/(k+1)
        print(cost.data.cpu().numpy())
        costs.append(cost)
        cost.backward()
        opt.step()
        #sched.step()uniform
        opt.zero_grad()
        if len(costs) > avgn + currn and len(costs)>min_iter:
            hist_costs = costs[-(avgn+currn):-currn]
            hist = sum(hist_costs).data[0] / avgn
            curr = sum(costs[-currn:]).data[0] / currn
            if abs((hist-curr)/hist) < eps/(2**k) or len(costs)>max_iter:
                break
        #print downsamples, updates - start_updates
      if write_field:
        self.stack.save('field', self.image_mip+k, self.bbox, field + I, is_field=True)
      if k > 0:
        field = upsample(field.permute(0,3,1,2)).permute(0,2,3,1)
    #print(cost.data[0], diff.data[0], penalty1.data[0])
    print('done:', updates)
    print(field.shape)
    # return self.center(field, (1,2), crop*2).data.cpu().numpy()[0]
    return self.center(field, (1,2), crop*2)

In [11]:
name = 'refactor_adjust_smoothness_v'
paths = {'image': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked',
         'fold': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1',
         'crack': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3',
         'loss': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/ground_truth',
         'output': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/barak_tests/{0}'.format(name),
         'field': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/barak_tests/{0}/field'.format(name)}
stack = Stack(paths)
offsets = [Vec(102716, 107077, 526), # fold
           Vec(102320, 111106, 526), # crack
           Vec(102565, 103904, 527)] # no defect
size = Vec(2048, 2048, 1)
image_mip = 2
fold_mip = 5
crack_mip = 5
loss_mip = 2
output_mip = 1
dst_increment = [-1,1]
for offset in offsets:
    bbox = Bbox(offset, offset+size)
    o = Optimizer(stack, bbox, dst_increment, image_mip, fold_mip, crack_mip, loss_mip, output_mip)
    field = o.process(ndownsamples=5, currn=5, avgn=20, lambda1=0.95, lr=0.1, 
                                                  eps=0.001, min_iter=100, max_iter=5000)
    o.render(field)

Downloading:   0%|          | 0/4 [00:00<?, ?it/s]
Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]


Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]



Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  8.87it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  8.05it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  6.46it/s]

Downloading:   0%|          | 0/4 [00:00<?, ?it/s]
Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]


Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]



Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]


Rechunking image: 100%|██████████| 1/1 [00:00<00:00, 11.78it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  9.19it/s]


Rechunking image: 100%|█████████

Downsample factor 16x
[0.26509356]
[0.25216794]
[0.24391137]
[0.2381247]
[0.23345363]
[0.2294529]
[0.22586489]
[0.22251058]
[0.21931538]
[0.21633324]
[0.2134082]
[0.2105998]
[0.20790991]
[0.20525154]
[0.20275205]
[0.20025375]
[0.19790265]
[0.19557858]
[0.1933866]
[0.19126667]
[0.18918002]
[0.18720315]
[0.18533632]
[0.18349862]
[0.18177389]
[0.18014568]
[0.17851622]
[0.17703012]
[0.17557198]
[0.17423189]
[0.17290592]
[0.17165586]
[0.17044452]
[0.16935328]
[0.16825828]
[0.16722417]
[0.1662341]
[0.16526464]
[0.16441467]
[0.16352242]
[0.16268343]
[0.16189848]
[0.16113359]
[0.16038409]
[0.15969473]
[0.15903494]
[0.15838003]
[0.15774503]
[0.15715933]
[0.15658428]
[0.1559818]
[0.1554379]
[0.15492922]
[0.15439802]
[0.15390533]
[0.15340894]
[0.15294115]
[0.15249562]
[0.15203366]
[0.15163463]
[0.15118977]
[0.15080647]
[0.1504142]
[0.15000317]
[0.14971831]
[0.14926581]
[0.14899713]
[0.14862601]
[0.14832765]
[0.1480124]
[0.14769556]
[0.14742121]
[0.14710858]
[0.14686973]
[0.14653498]
[0.14635627]


[0.13253975]
[0.13253307]
[0.13249692]
[0.13253635]
[0.13253708]


Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  7.28it/s]



Downsample factor 8x
[0.5648036]
[0.54198456]
[0.52227145]
[0.5050815]
[0.48965758]
[0.4760757]
[0.46367383]
[0.45495743]
[0.44560272]
[0.43885958]
[0.43189037]
[0.4256569]
[0.4203614]
[0.4153804]
[0.41163233]
[0.40748647]
[0.40405595]
[0.4010834]
[0.39807156]
[0.39539918]
[0.39348847]
[0.3911394]
[0.38926902]
[0.38779694]
[0.3856871]
[0.38490945]
[0.3829628]
[0.38180703]
[0.38029033]
[0.37834042]
[0.37809703]
[0.37609774]
[0.37538618]
[0.37457794]
[0.3729481]
[0.37302232]
[0.3717035]
[0.37115198]
[0.3706913]
[0.3697663]
[0.36961383]
[0.36882794]
[0.36859465]
[0.36768764]
[0.36758596]
[0.3669886]
[0.36667132]
[0.36641508]
[0.3656512]
[0.36582178]
[0.3651728]
[0.36499476]
[0.36476135]
[0.36390638]
[0.36437517]
[0.3636804]
[0.36345232]
[0.36323404]
[0.36276132]
[0.36237872]
[0.36231035]
[0.36207438]
[0.361462]
[0.3617398]
[0.36112943]
[0.36099726]
[0.36110628]
[0.36013842]
[0.36066175]
[0.3603132]
[0.35973835]
[0.36022228]
[0.35943246]
[0.3595925]
[0.3594567]
[0.35868037]
[0.35950166]
[0

...

Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  3.74it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  3.91it/s]



Downsample factor 1x
[1.6235738]
[1.2638676]
[1.2220011]
[1.2100435]
[1.2040881]
[1.2002147]
[1.1974132]
[1.1952106]
[1.1934245]
[1.1918097]
[1.1903805]
[1.189141]
[1.1880531]
[1.1870532]
[1.1860524]
[1.1852612]
[1.1844838]
[1.1837859]
[1.183064]
[1.182559]
[1.1819673]
[1.1814709]
[1.180942]
[1.1805041]
[1.1800005]
[1.1795651]
[1.1791245]
[1.1787772]
[1.1784005]
[1.1781237]
[1.1777108]
[1.1774807]
[1.1771226]
[1.1768184]
[1.1764741]
[1.1762636]
[1.1758976]
[1.175712]
[1.1754148]
[1.1752316]
[1.1749434]
[1.1747749]
[1.1745079]
[1.1743741]
[1.1741233]
[1.1739997]
[1.17377]
[1.1736648]
[1.1734616]
[1.1733124]
[1.1730691]
[1.1729796]
[1.1728005]
[1.1726396]
[1.1724789]
[1.1723633]
[1.1722151]
[1.1720662]
[1.1719108]
[1.1718256]
[1.1716788]
[1.1716199]
[1.1714687]
[1.171405]
[1.171268]
[1.1711757]
[1.1710788]
[1.1710283]
[1.1709163]
[1.1708529]
[1.1707458]
[1.1706588]
[1.1705327]
[1.1704497]
[1.1703324]
[1.1702726]
[1.1702006]
[1.1701233]
[1.1700497]
[1.1699908]
[1.169924]
[1.1698475]
[1.16

Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  3.71it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  3.04it/s]



done: 1385
torch.Size([1, 512, 512, 2])


Downloading:   0%|          | 0/4 [00:00<?, ?it/s]
Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]


Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  7.72it/s]



Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  5.76it/s]


Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  4.65it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  6.14it/s]

